### Imports

In [1]:
import sqlite3
import pyterrier as pt
import os
from pathlib import Path
from bs4 import BeautifulSoup  # To clean the html
import warnings
warnings.filterwarnings("ignore")

if not pt.started():
    pt.init()

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to C:\Users\AJS/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
C:\Users\AJS\AppData\Local\Temp\ipykernel_2784\2994591996.py:10: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


### SQLite DB for the Scraped Data

In [2]:
conn = sqlite3.connect("../indexation/scraper/deepmind_publications.db")  # Path to database

c = conn.cursor()

In [3]:
def extract_text(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text().replace('\n', '')

In [4]:
c.execute("SELECT title, abstract, url FROM publications")
documents = []
for title, abstract, url in c.fetchall():
    documents.append({"text": abstract, "title": title, "url": url})

In [5]:
documents[0]

{'text': 'We articulate fundamental mismatches between technical methods for machine unlearning in Generative AI, and documented aspirations for broader impact that these methods could have for law and policy. These aspirations are both numerous and varied, motivated by issues that pertain to privacy, copyright, safety, and more. For example, unlearning is often invoked as a solution for removing the effects of targeted information from a generative-AI model’s parameters, e.g., a particular individual’s personal data or in-copyright expression of Spiderman that was included in the model’s training data. Unlearning is also proposed as a way to prevent a model from generating targeted types of information in its outputs, e.g., generations that closely resemble a particular individual’s data or reflect the concept of “Spiderman.” Both of these goals—the targeted removal of information from a model and the targeted suppression of information from a model’s outputs—present various technical

In [6]:
documents[0]["text"]

'We articulate fundamental mismatches between technical methods for machine unlearning in Generative AI, and documented aspirations for broader impact that these methods could have for law and policy. These aspirations are both numerous and varied, motivated by issues that pertain to privacy, copyright, safety, and more. For example, unlearning is often invoked as a solution for removing the effects of targeted information from a generative-AI model’s parameters, e.g., a particular individual’s personal data or in-copyright expression of Spiderman that was included in the model’s training data. Unlearning is also proposed as a way to prevent a model from generating targeted types of information in its outputs, e.g., generations that closely resemble a particular individual’s data or reflect the concept of “Spiderman.” Both of these goals—the targeted removal of information from a model and the targeted suppression of information from a model’s outputs—present various technical and subs

### Indexing

In [ ]:
# TODO update indexing with new documents

In [ ]:
index_path = Path("../webscraper/index")
if os.path.exists(index_path):
    index = pt.IndexFactory.of(index_path)
else:
    indexer = pt.IterDictIndexer("../webscraper/index", meta={"docno": 20, "filename": 1024, "title": 1024}, meta_tags={"title":"title"})
    index_ref = indexer.index(documents)
    index = pt.IndexFactory.of(index_ref)

In [ ]:
print(index.getCollectionStatistics().toString())

In [ ]:
print("Available keys in MetaIndex:", index.getMetaIndex().getKeys())

In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25.transform("LLM")